*File Pickler*

This notebook is meant to be run locally to package up files into a list that is pickled and then encoded as a string.  This string is used by the solution's notebook to enable one-step deployment.

This workbook is not built to run in Snowflake.  The intent is to enable the person installing the solution to confirm that the pickle is as-described and safe to use.

In [3]:
import pickle
import codecs
import os
import base64
from snowflake.snowpark import Session
import json
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()

First, let's set our directories

In [4]:
# Set directories - separate text/code files and images
file_dir = os.getcwd() + "/toStage/streamlit/appPages"
image_dir = os.getcwd() + "/toStage/streamlit/Images"

In [5]:
# Methods for encoding and decoding objects
def encode(obj):
    pickled = pickle.dumps(obj)
    return codecs.encode(pickled, 'base64').decode()   

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

In [6]:
# Construct dictionaries to store all files and images
files_dict = {}
images_base64_dict = {}

# Iterate through all files in each directory and append contents to a list
for file in os.listdir(file_dir):
    with open(file_dir + "/" + file, "rb") as current_file:
        files_dict[file] = current_file.read()

for image in os.listdir(image_dir):
    with open(image_dir + "/" + image, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        images_base64_dict[image] = encoded_string

In [7]:
# Create serializeable object from lists
obj = {
    "files": files_dict,
    "images": images_base64_dict
}
# Encode object
encoded_obj = encode(obj)
# Print pickle string
#print(encoded_obj)
create_tbl_sql = """create or replace table dmm_test_db.code.file (
        NAME varchar
    ,   CONTENT varchar(16777216)
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}';"""
session.sql(create_tbl_sql).collect()
sql_string = f"""insert into dmm_test_db.code.file (NAME, CONTENT)
values ('PICKLE', '{encoded_obj}')"""
print(sql_string)
session.sql(sql_string).collect()

insert into dmm_test_db.code.file (NAME, CONTENT)
values ('PICKLE', '...
')


[Row(number of rows inserted=1)]

In [ ]:
# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

obj = decoded_obj
print(obj)